In [4]:
import requests
import json
import pandas as pd
import numpy as np
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import json_tuple,from_json,get_json_object

# Initialize spark
sc = SparkContext('local')
spark = SparkSession(sc)

In [31]:
# Custom function

def get_Uruguay_Users(location,page,per_page):

    # In case we call the API on Github, we need to login 
    # headers = {"app_id": "c7f6d128",
    #           "app_key": "73ea2ed8109721300050137e74044fa6"}
    url = "https://api.github.com/search/users?q=location:{location}&page={page}&per_page={per_page}".format(location=location,page=page,per_page=per_page)
    print(url)
    response = requests.get(url, headers={})
    return response

In [60]:
# Initialize everything

res_per_page = 100
current_page = 1
response = get_Uruguay_Users("Uruguay", current_page, res_per_page)
# print(response.text)

resultlist_json = json.loads(response.text)
df_meta = pd.DataFrame.from_dict(resultlist_json, orient='index')
# print(df_meta)
uu_df = df_meta[0]
total_count = uu_df["total_count"]
frames = [pd.DataFrame(data=uu_df["items"])]
fetched = len(uu_df["items"])

https://api.github.com/search/users?q=location:Uruguay&page=1&per_page=100
{"total_count":4020,"incomplete_results":false,"items":[{"login":"spastorino","id":52642,"node_id":"MDQ6VXNlcjUyNjQy","avatar_url":"https://avatars0.githubusercontent.com/u/52642?v=4","gravatar_id":"","url":"https://api.github.com/users/spastorino","html_url":"https://github.com/spastorino","followers_url":"https://api.github.com/users/spastorino/followers","following_url":"https://api.github.com/users/spastorino/following{/other_user}","gists_url":"https://api.github.com/users/spastorino/gists{/gist_id}","starred_url":"https://api.github.com/users/spastorino/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/spastorino/subscriptions","organizations_url":"https://api.github.com/users/spastorino/orgs","repos_url":"https://api.github.com/users/spastorino/repos","events_url":"https://api.github.com/users/spastorino/events{/privacy}","received_events_url":"https://api.github.com/users/spastori

In [76]:
total_count

4020

In [80]:
len(frames)

10

In [65]:
fetched

100

In [82]:
current_page

11

In [79]:

# get all pages until there are none left
while (fetched < total_count) and (fetched < 1000):
    
    current_page += 1
    response = get_Uruguay_Users("Uruguay", current_page, res_per_page)

    # WE NEED TO MAKE A PROCEDURE (or related) FOR THIS
    resultlist_json = json.loads(response.text)
    df_meta = pd.DataFrame.from_dict(resultlist_json, orient='index')
    uu_df = df_meta[0]
    total_count = uu_df["total_count"]
    frames.append(pd.DataFrame(data=uu_df["items"]))
    fetched += len(uu_df["items"])
    
    

In [81]:
resultlist_json

{'message': 'Only the first 1000 search results are available',
 'documentation_url': 'https://docs.github.com/v3/search/'}

In [74]:
result = pd.concat(frames)

In [88]:
result

login        id               node_id  \
0      spastorino     52642      MDQ6VXNlcjUyNjQy   
1       diegomura   5600341  MDQ6VXNlcjU2MDAzNDE=   
2            omab     91022      MDQ6VXNlcjkxMDIy   
3    pablohoffman    185212  MDQ6VXNlcjE4NTIxMg==   
4     lmgonzalves  12473993  MDQ6VXNlcjEyNDczOTkz   
..            ...       ...                   ...   
95          gab3d  18104091  MDQ6VXNlcjE4MTA0MDkx   
96       thepante  10443522  MDQ6VXNlcjEwNDQzNTIy   
97    petralorena  20385504  MDQ6VXNlcjIwMzg1NTA0   
98      agaleotti   6451595  MDQ6VXNlcjY0NTE1OTU=   
99  grazianobolla  35064738  MDQ6VXNlcjM1MDY0NzM4   

                                           avatar_url gravatar_id  \
0   https://avatars0.githubusercontent.com/u/52642...               
1   https://avatars1.githubusercontent.com/u/56003...               
2   https://avatars0.githubusercontent.com/u/91022...               
3   https://avatars1.githubusercontent.com/u/18521...               
4   https://avatars1.githubusercontent.com/u/12473...               
..                                                ...         ...   
95  https://avatars2.githubusercontent.com/u/18104...               
96  https://avatars2.githubusercontent.com/u/10443...               
97  https://avatars3.githubusercontent.com/u/20385...               
98  https://avatars0.githubusercontent.com/u/64515...               
99  https://avatars3.githubusercontent.com/u/35064...               

                                           url  \
0      https://api.github.com/users/spastorino   
1       https://api.github.com/users/diegomura   
2            https://api.github.com/users/omab   
3    https://api.github.com/users/pablohoffman   
4     https://api.github.com/users/lmgonzalves   
..                                         ...   
95          https://api.github.com/users/gab3d   
96       https://api.github.com/users/thepante   
97    https://api.github.com/users/petralorena   
98      https://api.github.com/users/agaleotti   
99  https://api.github.com/users/grazianobolla   

                            html_url  \
0      https://github.com/spastorino   
1       https://github.com/diegomura   
2            https://github.com/omab   
3    https://github.com/pablohoffman   
4     https://github.com/lmgonzalves   
..                               ...   
95          https://github.com/gab3d   
96       https://github.com/thepante   
97    https://github.com/petralorena   
98      https://github.com/agaleotti   
99  https://github.com/grazianobolla   

                                        followers_url  \
0   https://api.github.com/users/spastorino/followers   
1    https://api.github.com/users/diegomura/followers   
2         https://api.github.com/users/omab/followers   
3   https://api.github.com/users/pablohoffman/foll...   
4   https://api.github.com/users/lmgonzalves/follo...   
..                                                ...   
95       https://api.github.com/users/gab3d/followers   
96    https://api.github.com/users/thepante/followers   
97  https://api.github.com/users/petralorena/follo...   
98   https://api.github.com/users/agaleotti/followers   
99  https://api.github.com/users/grazianobolla/fol...   

                                        following_url  \
0   https://api.github.com/users/spastorino/follow...   
1   https://api.github.com/users/diegomura/followi...   
2   https://api.github.com/users/omab/following{/o...   
3   https://api.github.com/users/pablohoffman/foll...   
4   https://api.github.com/users/lmgonzalves/follo...   
..                                                ...   
95  https://api.github.com/users/gab3d/following{/...   
96  https://api.github.com/users/thepante/followin...   
97  https://api.github.com/users/petralorena/follo...   
98  https://api.github.com/users/agaleotti/followi...   
99  https://api.github.com/users/grazianobolla/fol...   

                                            gists_url  \
0   https://api.github.com/users/spas

In [89]:
result.to_csv('out.csv', index=False)  

In [113]:
##############################################################
# SOME OTHER APPROACHES USED, BUT USING PANDA IS WAY SIMPLER #
##############################################################

# The response from the server has some metadata, 
# like the total users, so we need to transform the data before using it 
# uu_df = df[0]
# uruguayUsers_df = uu_df["items"]
# print(uruguayUsers_df)


# This was a test to mannualy parse everything
# def get_list(x):
#    o_list = []
#    for elt in x:
#        o_list.append(pd.DataFrame.from_dict(elt, orient='index'))
#    return o_list
#
# list_df = get_list(uruguayUsers_df)
# list_df

# This was a test to mannualy parse everything
# li = [[item.get('id'), item.get('login')] for item in uruguayUsers_df]
# li

# df = pd.DataFrame(data=uruguayUsers_df)
# df